In [6]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [7]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [8]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(id='08d8659d-14b4-48cc-9e89-0269344a1dd6', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 3, 7, 41, 55, 372139, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

In [9]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

def get_city_price(city: str) -> str:
    """A search tool using Serper API to find airline ticket prices to specific cities.

    Args:
        city: The city to search for.
    """
    search_query = f"Airline prices for travel to {city}"
    return serper.run(search_query)

In [10]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [11]:
from autogen_core import CancellationToken

response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_nh8jgKPGobGMefJAPy9GQ4NE', arguments='{"city":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='Prices were available within the past 7 days and start at $137 for one-way flights and $364 for round trip, for the period specified. Prices and availability ... Book cheap flights to London (LHR) with United Airlines. Enjoy all the in-flight perks on your London flight, including speed Wi-Fi. Discover flights to United Kingdom. With fares from $456, we offer a great choice of food, drinks and onboard entertainment & WiFi. Find American Airlines flights from United States to England. Enjoy an exceptional travel experience at an affordable price. Book your trip today! Use Google Flights to find cheap flights to London from United States, starting at $363, and book tickets for your next getaway. Find flights to London (LON) from $184. Fly to London with Virgin Atlantic, TAP AIR PORTUGAL & more. New York from $184; Boston from $223; Newark from $229 ..

'Ah, London! Where the tea is strong and the accents are stronger. A roundtrip ticket will set you back about $364, or just enough to buy a whole lot of biscuits!☕✈️'